In [2]:
import pandas as pd
import csv
import xlsxwriter
import datetime
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
import os


now=datetime.datetime.now()
date = now.strftime("%d-%m-%Y")
time = now.strftime("%H:%M")

userpath=os.path.expanduser('~')


Input_file = os.path.join(userpath,'Desktop\\Inventory_raw.csv')
Inputdf=pd.read_csv(Input_file,'|')
Inputdf.columns = Inputdf.columns.str.replace('.',' ')
Inputdf.columns = Inputdf.columns.str.replace('_',' ')
Inputdf.columns=Inputdf.columns.str.strip()
Inputdf=Inputdf.dropna()
Inputdf['date'] = date
Inputdf['time'] = time

Inputdf['store inventories stock status']= Inputdf['store inventories stock status'].replace(3.0,"Available")
Inputdf['store inventories stock status']= Inputdf['store inventories stock status'].replace(0.0,"OOS")
Inputdf['products published']=Inputdf['products published'].str.strip()
Inputdf0=Inputdf[['product id','date','time','store inventories stock status','products published']]

#Published but OOS
OOS = Inputdf0['store inventories stock status']=="OOS"
Available = Inputdf0['store inventories stock status']=="Available"
Published = Inputdf0['products published']=="t"
Not_Published=Inputdf0['products published']=="f"

Inputdf2=Inputdf0[OOS&Published]
Inputdf3=Inputdf0[Available&Not_Published]

def pivot(inputdf):
    inputdf = inputdf.pivot_table(index=['date','time'],columns=['store inventories stock status'],values='product id',aggfunc='count')
    inputdf = inputdf.reset_index()
    return inputdf

Inputdf1=pivot(Inputdf0)
Inputdf2 = pivot(Inputdf2)
Inputdf3 = pivot(Inputdf3)

#Backend Catalog Quality
Inputdf1['Index']=(Inputdf1['Available']-Inputdf1['OOS'])/Inputdf1['Available']
Inputdf1['# of unpublished inventory']=Inputdf2['OOS']
Inputdf1['Published & OOS']=Inputdf3['Available']
Inputdf1['Visibility on Web']=1-Inputdf1['Published & OOS']/(Inputdf1['Published & OOS']+Inputdf1['Available'])
Inputdf1=Inputdf1[['date','time','Available','OOS','Index','# of unpublished inventory','Published & OOS','Visibility on Web']]

if os.path.isfile(os.path.join(userpath,'Desktop\\output.xlsx')) == True:
    #wb=load_workbook(r'C:\\Users\AT104\\Desktop\\output.xlsx')
    #sheet=wb.worksheets[0]
    #row_count=sheet.max_row
    #row_count=row_count+1
    wb = load_workbook(r'C:\\Users\AT104\\Desktop\\output.xlsx')
    ws=wb.get_active_sheet()
    for row in dataframe_to_rows(Inputdf1,index=False,header=False):
        ws.append(row)
        wb.save(os.path.join(userpath,'Desktop\\output.xlsx'))
        print('Successfully Appended, Congrats !')
else:
    save_excel=pd.ExcelWriter(os.path.join(userpath,'Desktop\\output.xlsx',engine='xlsxwriter'))
    Inputdf1.to_excel(save_excel,index=False,sheet_name="Sheet1")
    save_excel.save()
    print('New File Created, Where did you put your old file ?')    


Successfully Appended, Congrats !
